# Notebook Purpose:
## The main purpose of this notebook is to serve as a guide to train multiple top of the line models and combine their predictions.
## There will be NO EDA,
### for that check out this brilliant EDA by SERGEY SAHAROVSKIY here https://www.kaggle.com/code/sergiosaharovskiy/ps-s3e2-2023-eda-and-base-pytorch-model

# Imports

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [4]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

In [5]:
train.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


# Preprocessing

### First and foremost we'll check for any missing vales

In [6]:
pd.concat([train.isnull().sum().rename("missing_values_in_train"),
           test.isnull().sum().rename("missing_values_in_test")],
          axis=1)

,missing_values_in_train,missing_values_in_test
gender,0,0.0
age,0,0.0
hypertension,0,0.0
heart_disease,0,0.0
ever_married,0,0.0
work_type,0,0.0
Residence_type,0,0.0
avg_glucose_level,0,0.0
bmi,0,0.0
smoking_status,0,0.0


#### INSIGHTS: Phew, no missing values to deal with today!

### BEFORE continuing on with our preprocessing it's a good idea to combile both train and test datasets and them do preprocessing like encoding and stuff

In [7]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked
...,...,...,...,...,...,...,...,...,...,...
25503,Female,27.0,0,0,No,Private,Urban,75.77,17.6,never smoked
25504,Male,49.0,0,0,Yes,Private,Urban,102.91,26.7,Unknown
25505,Female,3.0,0,0,No,children,Rural,104.04,18.3,Unknown
25506,Male,31.0,0,0,Yes,Private,Urban,82.41,28.7,never smoked


### Identifying Categorical Features

In [8]:
pd.concat([df.nunique().rename("Unique Values"), df.dtypes.rename("Data Type")], axis=1).sort_values(by="Unique Values")

,Unique Values,Data Type
hypertension,2,int64
heart_disease,2,int64
ever_married,2,object
Residence_type,2,object
gender,3,object
smoking_status,4,object
work_type,5,object
age,109,float64
bmi,441,float64
avg_glucose_level,4345,float64


#### INSIGHTS: 
##### As we can see, most of the columns/features are categorical - in that they contain at most 2, 3 or 5 unique values.
##### And importantly, except for the hypertension and heart_disease columns, every other categorical column is of object/string type. And the reason for that is that both of these columns are already binary/one hot encoded as they contain either 0 or 1, indicating presence or absence of feature
#### So, we will NOT one hot encode hypertension and heart_disease cols, but will do the rest!

In [9]:
cat_cols = [col for col in df.columns if df[col].nunique() <= 5]
cat_cols

['gender',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'smoking_status']

In [10]:
# lets exclude hypertension and heart_disease as they are already encoded
cols_to_encode = list(set(cat_cols) - set(['hypertension', 'heart_disease']))
cols_to_encode

['gender', 'Residence_type', 'smoking_status', 'work_type', 'ever_married']

### Encoding Categorical Features
#### Since this data contains categorical features/columns, so our main preprocessing will concern with encoding them
#### Now, we could do this in two ways, either One Hot Encoding or Label/Oridinal Encoding
#### Here we will One Hote Encode but feel free to experiment with other techniques

In [11]:
# intialize the one hot encoder
one_hot_enc = OneHotEncoder(dtype="int64")

# fit it to the features we're interested in
one_hot_enc.fit(df[cols_to_encode])

# transform the features/columns and store the new columns back in the original dataset
# by default, the one_hot_enc.transform method returns a sparse matrix which we convert into numpy ndarray 
# and we use encoder's get features_names_out method passing it the cols we encoded to get the output features names for them. (i hope that made sense :p)
# see the cell next to see what I'm talking about
df[one_hot_enc.get_feature_names_out(cols_to_encode)] = pd.DataFrame(one_hot_enc.transform(df[cols_to_encode]).toarray())
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,...,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,ever_married_No,ever_married_Yes
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,...,0,1,0,0,0,1,0,0,0,1
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,...,1,0,0,0,0,1,0,0,0,1
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,...,0,0,0,0,0,1,0,0,0,1
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,...,0,1,0,0,0,1,0,0,0,1
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,...,0,1,0,0,0,1,0,0,1,0


In [12]:
# here's what this returns - the names of new cols/features that it automatically inferred based on the values in categorical cols that we wanted to encode
one_hot_enc.get_feature_names_out(cols_to_encode)

array(['gender_Female', 'gender_Male', 'gender_Other',
       'Residence_type_Rural', 'Residence_type_Urban',
       'smoking_status_Unknown', 'smoking_status_formerly smoked',
       'smoking_status_never smoked', 'smoking_status_smokes',
       'work_type_Govt_job', 'work_type_Never_worked',
       'work_type_Private', 'work_type_Self-employed',
       'work_type_children', 'ever_married_No', 'ever_married_Yes'],
      dtype=object)

#### Let's drop the original categorical columns that now have been encoded

In [13]:
df.drop(columns=cols_to_encode, axis=1, inplace=True)

In [14]:
# sanity check that the cols were dropped :D
len(df.columns)

21

#### Let's separate the train data from test data before we move on to the modeling phase

In [15]:
X = df.iloc[:-len(test), :]

test_new = df.iloc[-len(test):, :]

# the unprocessed and raw train dataframe that we loaded earlier
y = train.stroke

In [16]:
# sanity check
len(X) == len(train)

True

# Modeling

## Setting up Cross Validation
### Before we model, it's a good idea to define a cross validation function to cross validate our models

In [17]:
def cross_validate(X, y, model):
    kf = KFold(n_splits=5, shuffle=True, random_state=1337) # thumbs up if you're 1337 gang :D jk
    
    cv_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                
        # training
        model.fit(X_train, y_train, verbose=0)

        # predicting
        y_pred = model.predict_proba(X_val)[:, 1]
        
        auc = roc_auc_score(y_val, y_pred)
        
        print(f"Fold: {fold} \t auc: {auc}")
        
        cv_scores.append(auc)
    
    avg_auc = np.mean(cv_scores)
    print(f"Avg AUC: {avg_auc}")

## XGBoost

In [18]:
# randomly chosen parameters for now - make sure to tune your hyperparameters as they provide a significant boost!
xgb_params = {'n_estimators': 456,
 'max_depth': 8,
 'learning_rate': 0.0268,
 'min_child_weight': 10,
 'gamma': 0.22,
 'subsample': 0.62,
 'colsample_bytree': 0.33,
 'reg_alpha': 1.5211300201400934e-05,
 'reg_lambda': 0.0005497911242012908}

In [19]:
xgb_model = xgb.XGBClassifier(**xgb_params)

In [20]:
cross_validate(X, y, xgb_model)

Fold: 0 	 auc: 0.8886450204384986
Fold: 1 	 auc: 0.8719419115163795
Fold: 2 	 auc: 0.8828447182641516
Fold: 3 	 auc: 0.88107944081701
Fold: 4 	 auc: 0.8832801318875352
Avg AUC: 0.8815582445847149


## LightGBM

In [21]:
# randomly chosen params - make sure to tune yours!
lgbm_params = {'n_estimators': 10000,
                 'num_rounds': 291,
                 'learning_rate': 0.14293898453640025,
                 'num_leaves': 2780,
                 'max_depth': 8,
                 'min_data_in_leaf': 400,
                 'lambda_l1': 45,
                 'lambda_l2': 0,
                 'min_gain_to_split': 0.002584545158305085,
                 'bagging_fraction': 0.9,
                 'bagging_freq': 1,
                 'feature_fraction': 0.4}

In [22]:
lgbm_model = lgbm.LGBMClassifier(**lgbm_params)

In [23]:
cross_validate(X, y, lgbm_model)

[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] num_iterations is set=291, num_rounds=291 will be ignored. Current value: num_iterations=291
[LightGBM] [Warning] min_gain_to_split is set=0.002584545158305085, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.002584545158305085
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Fold: 0 	 auc: 0.8893025942958009
[LightGBM] [Warning]

## CatBoost

In [24]:
# randomly chosen params - make sure to tune yours!
catboost_params = {'loss_function': 'Logloss',
                     'learning_rate': 0.065,
                     'l2_leaf_reg': 0.43,
                     'colsample_bylevel': 0.083,
                     'depth': 5,
                     'min_data_in_leaf': 15,
                     'subsample': 0.73}

In [25]:
catboost_model = catboost.CatBoostClassifier(**catboost_params)

In [26]:
cross_validate(X, y, catboost_model)

Fold: 0 	 auc: 0.8846110298216276
Fold: 1 	 auc: 0.8562478892266125
Fold: 2 	 auc: 0.882784806334688
Fold: 3 	 auc: 0.8656454043194373
Fold: 4 	 auc: 0.8598034299167295
Avg AUC: 0.869818511923819


# Ensembling

In [45]:
def objective(trial):
    """Finding best weighted average params using optuna :D idea by: Khawaja Abaid (me) """
    w_xgb = round(trial.suggest_discrete_uniform("w_xgb", 0.0, 1.0, 0.1),1)
    w_lgbm = round(trial.suggest_discrete_uniform("w_lgbm", 0.0, float(round(1.0 - w_xgb, 1)), 0.1),1)
    w_cat = 1.0 - w_xgb - w_lgbm
    
    print(f"sum is {sum([w_xgb, w_lgbm, w_cat])}")
    
    if sum([w_xgb, w_lgbm, w_cat]) == 1.0:
    
        cv = KFold(n_splits=5, shuffle=True, random_state=1337)

        cv_scores = np.empty(5)
        for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            xgb_model = xgb.XGBClassifier(**xgb_params)
            xgb_model.fit(X_train, y_train, verbose=0)
            y_preds_xgb = xgb_model.predict_proba(X_test)[:, 1]

            lgbm_model = lgbm.LGBMClassifier(**lgbm_params)
            lgbm_model.fit(X_train, y_train, verbose=0)
            y_preds_lgbm = lgbm_model.predict_proba(X_test)[:, 1]

            catboost_model = catboost.CatBoostClassifier(**catboost_params)
            catboost_model.fit(X_train, y_train, verbose=0)
            y_preds_cat = catboost_model.predict_proba(X_test)[:, 1]

            y_preds_final = y_preds_xgb * w_xgb + y_preds_lgbm * w_lgbm + y_preds_cat * w_cat

            auc = roc_auc_score(y_test, y_preds_final)

            cv_scores[idx] = auc
        
        avg_auc = np.mean(cv_scores)
    
    # else if sum is not equal to 1.0, skip and for that we'll just reutn -99 as it will be the smallest value
    else:
        avg_auc = -99
    
    return avg_auc

In [46]:
study = optuna.create_study(study_name="weights", direction="maximize")
study.optimize(objective, n_trials=10)

[I 2023-01-10 16:54:17,935] A new study created in memory with name: weights


sum is 1.0
[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] num_iterations is set=291, num_rounds=291 will be ignored. Current value: num_iterations=291
[LightGBM] [Warning] min_gain_to_split is set=0.002584545158305085, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.002584545158305085
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=45, r

[I 2023-01-10 16:54:50,326] Trial 0 finished with value: 0.8824936248197561 and parameters: {'w_xgb': 0.9, 'w_lgbm': 0.0}. Best is trial 0 with value: 0.8824936248197561.


sum is 1.0
[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] num_iterations is set=291, num_rounds=291 will be ignored. Current value: num_iterations=291
[LightGBM] [Warning] min_gain_to_split is set=0.002584545158305085, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.002584545158305085
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=45, r

[I 2023-01-10 16:55:22,373] Trial 1 finished with value: 0.8834245785414824 and parameters: {'w_xgb': 0.1, 'w_lgbm': 0.6000000000000001}. Best is trial 1 with value: 0.8834245785414824.


...

[I 2023-01-10 16:58:38,069] Trial 7 finished with value: 0.8815582445847149 and parameters: {'w_xgb': 1.0, 'w_lgbm': 0.0}. Best is trial 5 with value: 0.8846031167809967.


sum is 1.0
[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] num_iterations is set=291, num_rounds=291 will be ignored. Current value: num_iterations=291
[LightGBM] [Warning] min_gain_to_split is set=0.002584545158305085, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.002584545158305085
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=45, r

[I 2023-01-10 16:59:10,831] Trial 8 finished with value: 0.8836082302164648 and parameters: {'w_xgb': 0.8, 'w_lgbm': 0.1}. Best is trial 5 with value: 0.8846031167809967.


sum is 1.0
[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] num_iterations is set=291, num_rounds=291 will be ignored. Current value: num_iterations=291
[LightGBM] [Warning] min_gain_to_split is set=0.002584545158305085, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.002584545158305085
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=45, r

[I 2023-01-10 16:59:42,908] Trial 9 finished with value: 0.8811756329240407 and parameters: {'w_xgb': 0.0, 'w_lgbm': 0.5}. Best is trial 5 with value: 0.8846031167809967.


In [47]:
study.best_value

0.8846031167809967

In [48]:
study.best_params

{'w_xgb': 0.6000000000000001, 'w_lgbm': 0.2}

## Keras

In [1]:
# we'll work with keras some other day

In [32]:
# inputs = layers.Input(shape=X.shape[1])

# x = layers.Dense(1024, activation="relu")(inputs)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.3)(x)

# x = layers.Dense(512, activation="relu")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.3)(x)

# x = layers.Dense(256, activation="relu")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.3)(x)

# x = layers.Dense(128, activation="relu")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.3)(x)

# outputs = layers.Dense(1, activation="sigmoid")(x)

# keras_model = keras.Model(inputs=inputs, outputs=outputs)
# keras_model.compile(optimizer="rmsprop",
#                    loss=keras.losses.binary_crossentropy,
#                    metrics=[keras.metrics.AUC()])

In [33]:
# early_stopping = keras.callbacks.EarlyStopping(
#                 patience=20,
#                 min_delta=0.001,
#                 monitor="val_auc",
#                 restore_best_weights=True,
#                 )

In [34]:
# sc = StandardScaler()
# X_scaled = sc.fit_transform(X)

In [35]:
# def keras_cv(X, y, model):
#     kf = KFold(n_splits=5, shuffle=True, random_state=1337) # thumbs up if you're 1337 gang :D jk
    
#     cv_scores = []
    
#     for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
#         X_train, X_val = X[train_idx], X[val_idx]
#         y_train, y_val = y[train_idx], y[val_idx]
        
#         history = model.fit(
#             X_train, y_train,
#             validation_data=(X_val, y_val),
#             batch_size=512,
#             epochs=50,
#             callbacks=[early_stopping],
#             )

#         y_pred = model.predict(X_val)
        
#         auc = roc_auc_score(y_val, y_pred)
        
#         print(f"Fold: {fold} \t auc: {auc}")
        
#         cv_scores.append(auc)
    
#     avg_auc = np.mean(cv_scores)
#     print(f"Avg AUC: {avg_auc}")


In [ ]:
# keras_cv(X_scaled, y, keras_model)